In [1]:
import cv2
import time
import numpy as np
import autopy
import ipynb.fs.full.handTrackingModule2  as htm

In [4]:
wCam,hCam = 640,480 
wScr,hScr = autopy.screen.size()
pTime =0
frameR =100
smoothening =7
pLocx,pLocy =0,0
print(wScr,hScr)

1536.0 864.0


In [3]:
cap =cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
detector = htm.HandDetector(maxHands =1)
while True:
    # 1. Find the hand landmarks
    success,img = cap.read()
    img = detector.findHands(img)
    lmList,bbox = detector.findPositions(img)
    
    # 2. Get the tip of the index and middle fingers
    if len(lmList)!=0:
        x1,y1 = lmList[8][1:]
        x2,y2 = lmList[12][1:]
        cv2.rectangle(img,(frameR,frameR),(wCam-frameR,hCam-frameR),(255,0,255),2)
        # print(x1,y1,x2,y2)
    # 3. chk which fingers are up
        fingers = detector.fingersUp()
        # print(fingers)
        
    # 4. Only index finger : Moving mode
        if fingers[1]==1 and fingers[2]==0:
            
        # 5. Convert coordinates
            
            x3 = np.interp(x1,(frameR,wCam-frameR),(0,wScr))
            y3 = np.interp(y1,(frameR,hCam-frameR),(0,hScr))
        
    # 6. Smooth Values
            cLocx = pLocx + (x3-pLocx)/smoothening
            cLocy = pLocy + (y3-pLocy)/smoothening
    # 7. Move Mouse
            autopy.mouse.move(wScr - cLocx,cLocy)
            cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
            pLocx,pLocy = cLocx,cLocy
    # 8. Both Inde and middle fingers are up : Clicking Mode
        if fingers[1]==1 and fingers[2]==1:
            
        # 9. Find distance between the fingers
            length,img,info = detector.findDistance(8,12,img)
            # print(length)
        # 10. clk mouse if distance is short
            if length <40:
                cv2.circle(img,(info[4],info[5]),15,(0,255,0),cv2.FILLED)
                autopy.mouse.click()
        
        
    cTime = time.time()
    fps = 1/(cTime -pTime)
    pTime =cTime
    
    cv2.putText(img,str(int(fps)) , (10,70),cv2.FONT_HERSHEY_PLAIN, 3,(255,0,255),3) # printing the fps
    cv2.imshow("Image",img)
    k = cv2.waitKey(1)
    if(k==27):
        break
cap.release()
cv2.destroyAllWindows()